In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from config import Config

c:\Users\HP\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LLMWrapper:
    def __init__(self):
        print(f"Loading LLM: {Config.MODEL_ID}...")
        self.tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_ID)
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = AutoModelForCausalLM.from_pretrained(
            Config.MODEL_ID,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        # Lưu ý: Nếu dùng device_map="auto", model đã tự vào đúng device, lệnh này thừa nhưng ko sao
        # self.model.to(device) 
        print("LLM Loaded successfully.")
    
    def generate(self, prompt, max_new_tokens=512):
        print("\n" + "="*50)
        print("BẮT ĐẦU QUÁ TRÌNH GENERATE")
        print("="*50)

        # --- BƯỚC 1: TẠO MESSAGES ---
        # Đây là cấu trúc chuẩn để model hiểu đâu là vai của AI, đâu là người dùng
        messages = [
            {"role": "system", "content": "Bạn là nhân viên hỗ trợ đặt món tại nhà hàng Hòa Viên."},
            {"role": "user", "content": prompt}
        ]
        print(f"\n[1] Messages (List of Dicts):\n{messages}")

        # --- BƯỚC 2: APPLY CHAT TEMPLATE ---
        # Biến đổi list trên thành 1 chuỗi văn bản duy nhất kèm các ký tự đặc biệt
        # Ví dụ model sẽ thêm <|im_start|>system...<|im_end|> tùy loại model
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        print(f"\n[2] Text sau khi Template (String raw):\n{text!r}") 
        # (Dùng !r để in ra cả các ký tự xuống dòng \n giúp dễ nhìn)

        # --- BƯỚC 3: TOKENIZATION ---
        # Biến đổi chữ thành số (Tensors)
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
        input_ids = model_inputs.input_ids
        print(f"\n[3] Input Ids (Tensor số hóa câu hỏi):\n{input_ids}")
        print(f" -> Kích thước Input: {input_ids.shape} (Dài {input_ids.shape[1]} tokens)")

        # --- BƯỚC 4: MODEL GENERATE ---
        # Máy tính toán và sinh ra các số tiếp theo
        print(f"\n[4] Đang suy nghĩ (Generate)...")
        generated_ids_raw = self.model.generate(
            model_inputs.input_ids,
            max_new_tokens=max_new_tokens
        )
        # generated_ids_raw lúc này chứa cả [Input cũ + Output mới]
        print(f" -> Generated Ids Raw (Gồm cả Input + Output):\n{generated_ids_raw}")
        print(f" -> Kích thước Raw: {generated_ids_raw.shape} (Dài {generated_ids_raw.shape[1]} tokens)")

        # --- BƯỚC 5: SLICING (CẮT BỎ INPUT) ---
        # Chỉ giữ lại phần mới sinh ra
        generated_ids_trimmed = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids_raw)
        ]
        # Vì batch_size=1 nên ta lấy phần tử đầu tiên xem cho dễ
        print(f"\n[5] Sliced Ids (Chỉ còn phần trả lời):\n{generated_ids_trimmed[0]}")
        
        # --- BƯỚC 6: DECODE ---
        # Dịch số ngược lại thành chữ
        response_text = self.tokenizer.batch_decode(generated_ids_trimmed, skip_special_tokens=True)[0]
        print(f"\n[6] Kết quả cuối cùng (Decoded Text):\n{response_text}")
        
        print("\n" + "="*50 + "\n")
        return response_text

In [3]:
bot = LLMWrapper()
bot.generate("Nhà hàng có món gì ngon?")

Loading LLM: Qwen/Qwen2.5-3B-Instruct...


c:\Users\HP\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--Qwen--Qwen2.5-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.50s/it]
Some parameters are on 

LLM Loaded successfully.

BẮT ĐẦU QUÁ TRÌNH GENERATE

[1] Messages (List of Dicts):
[{'role': 'system', 'content': 'Bạn là nhân viên hỗ trợ đặt món tại nhà hàng Hòa Viên.'}, {'role': 'user', 'content': 'Nhà hàng có món gì ngon?'}]

[2] Text sau khi Template (String raw):
'<|im_start|>system\nBạn là nhân viên hỗ trợ đặt món tại nhà hàng Hòa Viên.<|im_end|>\n<|im_start|>user\nNhà hàng có món gì ngon?<|im_end|>\n<|im_start|>assistant\n'

[3] Input Ids (Tensor số hóa câu hỏi):
tensor([[151644,   8948,    198,  94917,  37915, 128310,  97483, 129481, 128886,
         128845, 129994, 128272, 128275,  61758, 134315,  11401,  27021,     13,
         151645,    198, 151644,    872,    198,  79759,   6362,  61758,  28776,
         129994, 128382,   7777,    263,     30, 151645,    198, 151644,  77091,
            198]], device='cuda:0')
 -> Kích thước Input: torch.Size([1, 37]) (Dài 37 tokens)

[4] Đang suy nghĩ (Generate)...


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
c:\Users\HP\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


 -> Generated Ids Raw (Gồm cả Input + Output):
tensor([[151644,   8948,    198,  94917,  37915, 128310,  97483, 129481, 128886,
         128845, 129994, 128272, 128275,  61758, 134315,  11401,  27021,     13,
         151645,    198, 151644,    872,    198,  79759,   6362,  61758,  28776,
         129994, 128382,   7777,    263,     30, 151645,    198, 151644,  77091,
            198,     34,  21742, 128275,  61758, 134315,  11401,  27021, 128323,
         128907, 130249,  47742,   1319,    644, 140116,  69086,  78228, 129994,
         128442,     11,  94576, 129994, 129045, 128573, 128263,  78228, 129994,
         128442, 128688,     11, 137790,    489,  92358,  47742, 137790, 129597,
             13, 129005,  65526, 129994,   7777,    263, 129144, 132345, 128260,
           1447,     16,     13,    386,   3165,    521,    352,     25,    425,
         124415, 135570,     11, 132710,    856,   4458,     78,    521,    352,
             11, 132710,    296,  23531,    521,    352,   933

'Cả nhà hàng Hòa Viên rất đa dạng và phong phú trong các món ăn, từ món truyền thống đến các món ăn sáng, bữa trưa và bữa tối. Một số món ngon nổi bật như:\n\n1. Món chay: Bánh cuốn, bánh xèo chay, bánh mì chay...\n2. Món thịt: Bún chả, bún riêu cua, bún cá...\n3. Món đặc sản: Bún chả cá, bún riêu cua, bún đậu mắm tôm...\n4. Món ăn sáng: Bánh mì trứng, bún bò Huế, phở...\n\nTất cả đều được chế biến với nguyên liệu tươi ngon và cách trình bày đẹp mắt. Mỗi món ăn đều mang hương vị riêng biệt của nhà hàng, phù hợp với khẩu vị nhiều người. Bạn hãy liên hệ trực tiếp để được tư vấn thêm về thực đơn hôm nay nhé!'